In [1]:
library(ggplot2)
library(nlme)
library(lme4)
library(dplyr)
library(multcomp)
library(english)


Warning message:
"package 'ggplot2' was built under R version 3.6.3"Warning message:
"package 'lme4' was built under R version 3.6.3"Loading required package: Matrix

Attaching package: 'lme4'

The following object is masked from 'package:nlme':

    lmList

Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following object is masked from 'package:nlme':

    collapse

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'multcomp' was built under R version 3.6.3"Loading required package: mvtnorm
Warning message:
"package 'mvtnorm' was built under R version 3.6.3"Loading required package: survival
Loading required package: TH.data
Loading required package: MASS

Attaching package: 'MASS'

The following object is masked from 'package:dplyr':

    select


Attaching package: 'TH.data'

The following 

In [7]:
# setwd('D:/Google Drive/Yan Experiment')

exp_name = '4t_180trial_4block'

setwd(paste('../',exp_name,sep=''))
test = read.csv(paste('vigor_conf_',exp_name,'.csv',sep=''))

In [8]:
#====================== Filter and add Probs ================
# famil_data = filter(test,trial<=16)
# prefilt_data = filter(test,trial<=(16+36))
# test = filter(test,trial>16)
test = test[test$trial >= 17,]

trials = c()
counter = 1
subj = 1
for (k in 1:length(test$trial)){
  if (test[k,]$subj!=subj){
    counter = 1
    subj = test[k,]$subj
  }
  trials = c(trials,counter)
  if (test[k,'diff_prob']>0){
    test[k,'diff_prob'] = test[k,'diff_prob']-.01
  }
  counter = counter + 1
}
test$trial_in_block= (trials-1)%%180+1

In [4]:
blocks = c()
counter = 1
subj = 1
block = 1

for (k in 1:length(test$trial)){
  if (test[k,]$subj!=subj){
    subj = test[k,]$subj
    block = 1
  }
  blocks = c(blocks,block)
  if (test[k,]$trial_in_block == 180){
    block = block + 1
  }
}
test$block = blocks

In [5]:

add_n_back <- function(data,n_back){
  data[1,paste(as.english(n_back),'_back_rate',sep='')] = 0
  data[1,paste(as.english(n_back),'_back_rate_tar',sep='')] = 0
  for (k in 2:length(data$subj)){
    subject = data[k,'subj']
    block = data[k,'block']
    cur_tar = data[k,'target']
    rate = 0
    tar_rate = 0
    nb = 0
    last_back = k-n_back
    if (last_back <= 0){
      last_back = 1
    }
    for (j in (k-1):last_back){
      if (data[j,'subj'] != subject | data[j,'block'] != block){
        # print(paste('Breaking at ',j,sep=''))
        break
      }
      nb = nb + 1
      rate = rate + (.5^(nb))*data[j,'rewarded']
      
      if (data[j,'target'] == cur_tar){
        tar_rate = tar_rate + (.5^(nb))*data[j,'rewarded']
      } else{
        tar_rate = tar_rate + (.25^(nb))*data[j,'rewarded']
      }
    }
    data[k,paste(as.english(n_back),'_back_rate',sep='')] = rate
    data[k,paste(as.english(n_back),'_back_rate_tar',sep='')] = tar_rate
  }
  return(data)
}

for (nbacks in c(1)){#},3,5,10)){
  test <- add_n_back(test,nbacks)
}

In [6]:
summary(lme(peak_vel~r_prob+as.factor(target), random =~1|subj, data=test))

Linear mixed-effects model fit by REML
 Data: test 
        AIC       BIC   logLik
  -34517.95 -34464.92 17265.97

Random effects:
 Formula: ~1 | subj
        (Intercept)   Residual
StdDev:   0.1538439 0.07243113

Fixed effects: peak_vel ~ r_prob + as.factor(target) 
                        Value  Std.Error    DF   t-value p-value
(Intercept)         0.4962583 0.03443116 14376  14.41306  0.0000
r_prob              0.0024856 0.00162119 14376   1.53323  0.1252
as.factor(target)2 -0.0562279 0.00170722 14376 -32.93537  0.0000
as.factor(target)3 -0.0211226 0.00170722 14376 -12.37253  0.0000
as.factor(target)4 -0.0428930 0.00170722 14376 -25.12448  0.0000
 Correlation: 
                   (Intr) r_prob as.()2 as.()3
r_prob             -0.023                     
as.factor(target)2 -0.025  0.000              
as.factor(target)3 -0.025  0.000  0.500       
as.factor(target)4 -0.025  0.000  0.500  0.500

Standardized Within-Group Residuals:
         Min           Q1          Med           Q3   

In [7]:
cftest(lmer(peak_vel~r_prob+as.factor(target)+(1|subj), data=test))


	 Simultaneous Tests for General Linear Hypotheses

Fit: lmer(formula = peak_vel ~ r_prob + as.factor(target) + (1 | subj), 
    data = test)

Linear Hypotheses:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept) == 0         0.496258   0.034431  14.413   <2e-16 ***
r_prob == 0              0.002486   0.001621   1.533    0.125    
as.factor(target)2 == 0 -0.056228   0.001707 -32.935   <2e-16 ***
as.factor(target)3 == 0 -0.021123   0.001707 -12.373   <2e-16 ***
as.factor(target)4 == 0 -0.042893   0.001707 -25.124   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
(Univariate p values reported)
